# Global and local scopes

In Python the **global** scope refers to the **module** scope.

The scope of a variable is normally defined by **where** it is (lexically) defined in the code.

In [1]:
a = 10
def my_func(n):
    print('global:', a)
    c = a ** n
    return c
my_func(5)

global: 10


100000

a is a global variable, and c is a local variable.

The scope of a variable is determined by where it is assigned. In particular, any variable defined (i.e. assigned a value) inside a function is local to that function, even if the variable name happens to be global too!

Remember that whenever you assign a value to a variable without having specified the variable as **global**, it is **local** in the current scope. **Moreover**, it does not matter **where** the assignment in the code takes place, the variable is considered local in the **entire** scope - Python determines the scope of objects at compile-time, not at run-time.

# Nonlocal scopes

Functions defined inside anther function can reference variables from that enclosing scope, just like functions can reference variables from the global scope.

In [2]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        print(x)
    
    inner_func()
    print(x)

outer_func()

hello
hello


In [3]:
# The x variable in inner_func is a local variable which won't affect 
# the x in the outer_func scope.
def outer_func():
    x = 'hello'
    
    def inner_func():
        x = 'good'
        print(x)
    
    inner_func()
    print(x)

outer_func()

print('====nonlocal====')
# But we can change the x in the outer_func scope by adding nonlocal to the inner_func
def outer_func():
    x = 'hello'
    
    def inner_func():
        nonlocal x
        x = 'good'
        print(x)
    
    inner_func()
    print(x)

outer_func()


good
hello
====nonlocal====
good
good


# Closures

Closure: **A function** plus an **extended scope** that contains the free variables.

In [4]:
# The function outer returns a closure inner
def outer():
    x = 'python'
    print('outer x: ', hex(id(x)))
    def inner():
        print('inner x: ', hex(id(x)))
        print(x)
    return inner
fn = outer()

outer x:  0x7f4dd366d170


In [5]:
fn()

inner x:  0x7f4dd366d170
python


In [6]:
fn

<function __main__.outer.<locals>.inner()>

In [7]:
fn.__closure__

(<cell at 0x7f4dd0631790: str object at 0x7f4dd366d170>,)

The closure is an reference pointing to a string object, which is the x, and the address of x in inner and outer are the same.

## Applications

In [8]:
# An average function

def average():
    """
    Returns the average number as you add numbers.
    """
    total = 0
    count = 0
    def add(n):
        nonlocal total, count
        total += n
        count += 1
        return total/count
    return add

In [9]:
avg = average()

In [10]:
avg(10)

10.0

In [11]:
avg(20)

15.0

In [12]:
# A timer example
from time import perf_counter

In [13]:
def itmer():
    start = perf_counter()
    def elapsed():
        return perf_counter() - start
    return elapsed

In [14]:
t = itmer()

In [15]:
t()

0.004775549998157658

In [16]:
t()

0.010409366001113085

In [17]:
# A counter function
def counter(init_value=0):
    def inc(step=1):
        nonlocal init_value
        init_value += step
        return init_value
    return inc

In [18]:
c = counter(2)

In [19]:
for i in range(10):
    print(f'Running {c(2)} times')

Running 4 times
Running 6 times
Running 8 times
Running 10 times
Running 12 times
Running 14 times
Running 16 times
Running 18 times
Running 20 times
Running 22 times


In [20]:
# Build a function to count how many times we have run some function.
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print(f'Running the {fn.__name__} {count} times')
        return fn(*args, **kwargs)
    return inner

In [21]:
def add_(a, b):
    return a + b

In [22]:
add_ = counter(add_)
add_(5, 6)

Running the add_ 1 times


11

In [23]:
add_(100, 100)

Running the add_ 2 times


200

In [24]:
# Modify the counter function to record run count for whatever function we pass in.
def counter(fn, counters):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        counters[fn.__name__] = count
        return fn(*args, **kwargs)
    return inner

In [25]:
def add(a, b):
    """
    Add two numbers or two strings.
    """
    return a + b

def mult(a, b):
    """
    Multiple two numbers or a string with a number.
    """
    return a * b

In [26]:
func_counters = {}
count_add = counter(add, func_counters)
count_mult = counter(mult, func_counters)

In [27]:
print(count_add(5,6))
print(count_add('a', 'b'))
print(count_mult(2, 3))
print(count_mult('a', 5))

11
ab
6
aaaaa


In [28]:
func_counters

{'add': 2, 'mult': 2}

# Decorators

In [29]:
def counter(fn):
    count = 0
    
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [30]:
@counter
def add(a, b):
    """
    returns sum of two integers
    """
    return a + b

In [31]:
add(4, 4)

Function add was called 1 times


8

In [32]:
add(2, 3)

Function add was called 2 times


5

In [33]:
add.__name__

'inner'

The name of the function is no longer add, but instead it's the name of the inner function.

We can use wraps to make everything back to normal.

In [34]:
from functools import wraps

In [35]:
def counter(fn):
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

@counter
def add(a, b):
    """
    returns sum of two integers
    """
    return a + b

In [36]:
add(4, 4)

Function add was called 1 times


8

In [37]:
print(add.__name__)
print(add.__doc__)

add

    returns sum of two integers
    


In [38]:
help(wraps)

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



## Decorator application - Timer

Create a function to time how long a function takes to run a certain function.

In [39]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__, args_str, elapsed))
        return result
    
    return inner

Let's write a function that calculates the n-th Fibonacci number:

`1, 1, 2, 3, 5, 8, ...`

We will implement this using two different methods:
1. recursion
2. a loop

In [40]:
# Recursion
def fib_rec(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_rec(n-1) + fib_rec(n-2)
    
# Only time the recursion function with final result.
# It prevents from printing all recursive steps.
@timed
def fib_recursed(n):
    return fib_rec(n)

In [41]:
fib_recursed(35)

fib_recursed(35) took 2.136188s to run.


9227465

In [42]:
# Loop

def fib_loop(n):
    f0 = 0
    f1 = 1
    for i in range(2, n+1):
        f0, f1 = f1, f1 + f0
    return f1

In [43]:
fib_loop(10)

55

In [44]:
@timed
def fib_looped(n):
    return fib_loop(n)

In [45]:
fib_looped(35)

fib_looped(35) took 0.000007s to run.


9227465

The loop method is more faster. That's because the recursive one will re-calculate intermediate variables every time.

## Decorator application-Logger, Stacked decorators

 we're going to create a utility decorator that will log function calls.

In [46]:
def logger(fn):
    from functools import wraps
    from datetime import datetime
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now()
        result = fn(*args, **kwargs)
        print(f'{fn.__name__}: called {run_dt}.')
        return result
    return inner

def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__, args_str, elapsed))
        return result
    
    return inner

Apply two decorators.

In [47]:
@logger
@timed
def func1():
    pass

In [48]:
func1()

func1() took 0.000000s to run.
func1: called 2020-05-16 15:40:00.050087.


In [49]:
@timed
@logger
def func1():
    pass

In [50]:
func1()

func1: called 2020-05-16 15:40:00.061265.
func1() took 0.000086s to run.


The order of the decorator may matters in practice.

For example:
    
    If you decorated it this way:

<pre>
@log
@authorize
def my_endpoint():
    pass
</pre>

then the call would always be logged.

But, in this instance:

<pre>
@authorize
@log
def my_endpoint():
    pass
</pre>

your endpoint would only get logged if the user passed the `authorize` test.

## Memorization

In the previous fibonacci example, the running time is pretty longer if we use recursive method. 

When we run the recursive fibonacci, we see that it is quite inefficient, as the same Fibonacci numbers get calculated multiple times:

In [51]:
def fib(n):
    print(f'Calculating fib({n}).')
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [52]:
fib(4)

Calculating fib(4).
Calculating fib(3).
Calculating fib(2).
Calculating fib(1).
Calculating fib(0).
Calculating fib(1).
Calculating fib(2).
Calculating fib(1).
Calculating fib(0).


3

It would be better if we could store these results. If we have calculated fib(n) before, we can simply recall these values instead of recalculating them.

In [53]:
# Use a closure to achieve it.
def fib():
    cache = {0:0, 1:1}
    
    def inner(n):
        if n in cache:
            return cache[n]
        print(f'Calculating fib({n})')
        result = inner(n - 1) + inner(n - 2)
        cache[n] = result
        return result
    return inner

In [54]:
fib_func = fib()

In [55]:
fib_func(4)

Calculating fib(4)
Calculating fib(3)
Calculating fib(2)


3

In [56]:
fib_func(5)

Calculating fib(5)


5

In [57]:
# Implement using a decorator.
def memorize(fn):
    cache = {}
    
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    return inner

In [58]:
@memorize
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [59]:
fib(50)

12586269025

In [60]:
fib(51)

20365011074

The functools module has a function lru_cache which is more efficient compared to the rudimentary memoization example we did above.

LRU Cache = Least Recently Used caching: since the cache is not unlimited, at some point cached entries need to be discarded, and the least recently used entries are discarded first

In [61]:
from functools import lru_cache

@lru_cache()
def fib(n):
    print(f'Calculating fib({n}).')
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [62]:
fib(35)

Calculating fib(35).
Calculating fib(34).
Calculating fib(33).
Calculating fib(32).
Calculating fib(31).
Calculating fib(30).
Calculating fib(29).
Calculating fib(28).
Calculating fib(27).
Calculating fib(26).
Calculating fib(25).
Calculating fib(24).
Calculating fib(23).
Calculating fib(22).
Calculating fib(21).
Calculating fib(20).
Calculating fib(19).
Calculating fib(18).
Calculating fib(17).
Calculating fib(16).
Calculating fib(15).
Calculating fib(14).
Calculating fib(13).
Calculating fib(12).
Calculating fib(11).
Calculating fib(10).
Calculating fib(9).
Calculating fib(8).
Calculating fib(7).
Calculating fib(6).
Calculating fib(5).
Calculating fib(4).
Calculating fib(3).
Calculating fib(2).
Calculating fib(1).
Calculating fib(0).


9227465

In [63]:
fib(37)

Calculating fib(37).
Calculating fib(36).


24157817

## Pass variables to a decorator

In [64]:
def timed(reps):
    def decorator(fn):
        @wraps(fn)
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_elapsed = total_elapsed / reps
            print(f'The average running time is {avg_elapsed}s for {reps} reps.')
            return result
        return inner
    return decorator

In [65]:
def fib_rec(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib_rec(n-1) + fib_rec(n-2)

In [66]:
@timed(10)
def fib_recursive(n):
    return fib_rec(n)
fib_recursive(20)

The average running time is 0.0016835738002555444s for 10 reps.


6765

In [67]:
@timed(5)
def fib_recursive(n):
    return fib_rec(n)
fib_recursive(20)

The average running time is 0.0016550772001210135s for 5 reps.


6765

## Decorator application-Decorator class

In [68]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f'MyClass instance called: a={self.a}, b={self.b}.')
            return fn(*args, **kwargs)
        return inner

In [69]:
@MyClass(20, 40)
def add(a, b):
    return a + b
add(10, 10)

MyClass instance called: a=20, b=40.


20

## Decorator application: Decorating Classes

In [70]:
def dec_speak(cls):
    cls.speak = lambda self:'This is a test!'
    return cls

In [71]:
@dec_speak
class Test:
    def __init__(self):
        self.len = 0

In [72]:
t = Test()

In [73]:
t.speak()

'This is a test!'

### A debug example

In [74]:
from datetime import datetime
def debug_info(cls):
    def info(self):
        results = []
        results.append(f'time: {datetime.now()}')
        results.append(f'class: {self.__class__.__name__}')
        if self.__dict__:
            results.extend('{}: {}'.format(k, v)  \
                           for k, v in self.__dict__.items())
        return results
    cls.debug = info
    return cls

In [75]:
@debug_info
class Car:
    def __init__(self, length, odometer=0):
        self.len = length
        self.odo = odometer
    
    def drive(self):
        self.odo += 1

In [76]:
c = Car(18)
c.drive()

In [77]:
c.debug()

['time: 2020-05-16 15:40:00.230073', 'class: Car', 'len: 18', 'odo: 1']

## Decorator application: Single Dispatch

Build a function to display various data types in html format, with different presentations for integers, floats, strings, lists and tuples should be implemented using bulleted lists, and the same with dictionaries except we want the name/value pair to be displayed in the bulleted list.

In [94]:
from html import escape

In [98]:
# An escape example
escape('&apple <go >')

'&amp;apple &lt;go &gt;'

In [154]:
# Implement a singledispatch function
def singledispatch(fn):
    registry ={}
    registry[object] = fn
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
        return inner
    
    def decorator(arg):
        result = registry.get(type(arg), registry[object])(arg)
        return result
    decorator.register = register
    decorator.registry = registry
    return decorator

In [155]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [156]:
htmlize('a < 100')

'a &lt; 100'

In [157]:
htmlize(50)

'50'

In [158]:
# Add the html_int function for type int
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

In [159]:
# Now we can render the int.
htmlize(50)

'50(<i>0x32</i)'

In [161]:
# And we can look at our registry to see what functions had been included.
htmlize.registry

{object: <function __main__.htmlize(a)>, int: <function __main__.html_int(a)>}

In [162]:
# Use the functools module
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence

In [163]:
@singledispatch
def htmlize(a):
    return escape(str(a))

The `singledispatch` returned closure has a few attributes we can use:
1. A `register` decorator (just like ours did)
2. A `registry` property that is the registry dictionary
3. A `dispatch` function that can be used to determine which registry key (registered type) it will use for the specified type.

In [164]:
@htmlize.register(Integral)
def htmlize_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a))) 

In [165]:
htmlize(5)

'5(<i>0x5</i)'

In [166]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_int(a)>})

In [168]:
htmlize.dispatch(int)

<function __main__.htmlize_int(a)>